<a href="https://colab.research.google.com/github/Gaurav7004/RSE_TOOL/blob/main/RSE_TOOL_Balanced_Half_Sample_Method_01_06_2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ! pip install pandarallel
# from pandarallel import pandarallel
import json
import re
import pandas as pd
import numpy as np
import itertools

In [2]:
# pandarallel.initialize()

In [3]:
df = pd.read_csv('Data1.csv')

# remove blank spaces from front and back of column names
df.columns = df.columns.str.strip()
df.columns

<ipython-input-3-fc8b3e05b5c4>:1: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('Data1.csv')


Index(['SEC', 'ST', 'FSU', 'IDNO', 'SEG', 'SSS', 'SSU', 'STRM', 'SSTRM', 'SS',
       'NSS', 'NSC', 'MULT', 'HH_SIZE', 'HHTYPE', 'RELG', 'SG', 'MHCE',
       'DC_ALL', 'DC_ST', 'SRL', 'REL', 'SEX', 'MSEX', 'AGE', 'MARST', 'GEDU',
       'PAS', 'PS_SS', 'CWS', 'ERN_SELF', 'ERN_REG'],
      dtype='object')

In [4]:
## Data is the dictionary
data = {}

## Keys
key = ''

## values
values = []

## Open
with open('Example1.RSE', "r") as f:
    for line in f:
        line = line.strip()
        if line.startswith('#'):
            if key:
                key = key.split("'")[0]
                key = key.split("\t")[0]
                data[key] = [v for v in values if v and not v.startswith("'")]
            key = line.split("'")[0]
            key = key.split("\t")[0]
            values = []
        elif line.startswith("'"):
            pass
        else:
            line = line.split("'")[0]
            values.append(line.strip())

# add last group
if key:
    data[key] = [v for v in values if v and not v.startswith("'")]

print(json.dumps(data, indent=4))

{
    "#SAMPLING SUBSAMPLE2": [],
    "#GROUP 2": [
        "SEC   C sec",
        "ST_GR C NEWVAR"
    ],
    "#FILTER 1": [
        "AGE>=15"
    ],
    "#VARIABLE 3  ": [
        "POP\tN  NEWVAR",
        "LF\tN  NEWVAR",
        "WRK\tN  NEWVAR"
    ],
    "#FILE Data1.Xlsx": [],
    "#RENAME  5    ": [
        "STRMID C sec+st+strm+sstrm",
        "SS     C ss",
        "NSS    N  nss",
        "NSC    N  nsc",
        "MULT   N  mult"
    ],
    "#TRANSFORM 4  ": [
        "ST_GR=(\"G1\")   ST in (\"10,19,20,21\")",
        "ST_GR=(\"G2\")   ST in (\"11,12,13,14,15,16,17,18\")",
        "ST_GR=(\"G3\")   Otherwise",
        "POP=(1)",
        "WRK=(1)  CWS in (\"11,12,21,31,41,51,61,61,71,72\")",
        "WRK=(0)  CWS NOT in (\"11,12,21,31,41,51,61,61,71,72\")",
        "LF=(1)   CWS in (\"11,12,21,31,41,51,61,61,71,72,81\")",
        "LF=(0)   Otherwise"
    ],
    "#EST_RSE 7": [
        "S POP",
        "E POP",
        "R POP",
        "E 100*LF/POP",
        "R LF/POP",
    

In [5]:
def SAMPLING_(data, df):
    for key, value in data.items():
        ## To read the sampling method used
        if str('SAMPLING').lower() in str(key).lower():
            if str('SUBSAMPLE2').lower() in str(key).lower():
                return str('SUBSAMPLE2')

            elif str('SRSWR').lower() in str(key).lower():
                pass
            elif str('SRSWOR').lower() in str(key).lower():
                pass

SAMPLING_(data, df)

'SUBSAMPLE2'

In [6]:
df.columns

Index(['SEC', 'ST', 'FSU', 'IDNO', 'SEG', 'SSS', 'SSU', 'STRM', 'SSTRM', 'SS',
       'NSS', 'NSC', 'MULT', 'HH_SIZE', 'HHTYPE', 'RELG', 'SG', 'MHCE',
       'DC_ALL', 'DC_ST', 'SRL', 'REL', 'SEX', 'MSEX', 'AGE', 'MARST', 'GEDU',
       'PAS', 'PS_SS', 'CWS', 'ERN_SELF', 'ERN_REG'],
      dtype='object')

In [7]:
### ---------------------------------------------------------------------------------------------------------
def GROUP_(data, df):
    for key, value in data.items():    
        if str('GROUP').lower() in str(key).lower():
            value = [s.split()[0] for s in value]
            for name in value:
                if name in df.columns:
                    pass
                elif name not in df.columns:
                    df[name] = 0

            return value


    for key, value in data.items():
        print(key, value)
        ### To read the group columns
        if str('GROUP').lower() in str(key).lower():
            # Define regular expression to match three words separated by whitespace
            regex = r'\w+\s+\w+\s+\w+'

            for i in range(len(value)):
                # Find the first match in the string
                match = re.search(regex, value[i])

                # Print the match
                lst = match.group().split()

                print(lst[0], lst[1])

                if lst[0] in df.columns:
                    if lst[1] == 'C':
                        df[lst[0]] = df[lst[0]].astype(str)
                elif lst[0] not in df.columns:
                    # if lst[1] == 'C':
                    #     df[lst[0]] = ''
                    pass

        else:
            pass

### ---------------------------------------------------------------------------------------------------------
def FILTER_(data, df):
    lst_queries = []
    for key, value in data.items():

        if str('FILTER').lower() in str(key).lower():
            # Define regular expression to match three words separated by whitespace
            regex = r'\([^()]+\)|\w+[<>!=]+[0-9]+(?:\s*(?:&&|\|\|)\s*(?:\([^()]+\)|\w+[<>!=]+[0-9]+))*'

            for i in range(len(value)):
                # Find the first match in the string
                match = re.search(regex, value[i])

                # Print the match
                lst_queries.append(match.group())

        else:
            pass

    filter_query = ' & '.join(['({})'.format(item) for item in lst_queries])
   
    try:
        df = df.query(filter_query)
        return df
    except:
        return df


### ---------------------------------------------------------------------------------------------------------
def FILE_(data):
    for key, value in data.items():
        ### To read the file used
        if str('FILE').lower() in str(key).lower():
            if key.strip().endswith('.xlsx') or key.strip().endswith('.csv'):
                match = re.search(r'[A-Za-z0-9]+\.xlsx|[A-Za-z0-9]+\.csv', key)   
                if match:
                    file_name = match.group(0)
                    return file_name
                else:
                    file_name = None
                    return file_name
                print(file_name)

            # return file_name

### ---------------------------------------------------------------------------------------------------------
def RENAME_(data, df):
    # create an empty dictionary to store the column names
    column_dict = {}

    for key, value in data.items():
        ### To read the file used
        if str('RENAME').lower() in str(key).lower():
            # value = [elem for elem in value[0].split(' ') if elem.strip()]
            # return value

            for col in value:
                col_split = col.split()

                # print(col_split[0])

                # extract the last element as the column name
                col_name = col_split[-1]

                new_col_name = col_split[0]

                ### if there is a "+" sign in the last element, concatenate the columns
                if "+" in col_name:
                    col_concat = col_name.split("+")
                    col_concat = [i.upper() for i in col_concat]

                    # concatenate the columns
                    # df[new_col_name] = df.loc[:, col_concat].apply(lambda x: ''.join(['0' + str(i) if i < 10 else str(i) for i in x]), axis=1)
                    # df[new_col_name] = df.loc[:, col_concat].apply(lambda x: ''.join(['0' + str(i) if i < 10 else str(i) for i in x]), axis=1)
                    df[new_col_name] = df.loc[:, col_concat].apply(lambda x: ''.join(['0' + str(i) if int(i) < 10 else str(i) for i in x]), axis=1)


                else:
                    col_name = col_name.upper()

### ---------------------------------------------------------------------------------------------------------
def NEWVARIABLE_(data, df):
    for key, value in data.items():
        ### To read the file used
        if str('VARIABLE').lower() in str(key).lower():
            value = [s.split()[0] for s in value]
            
            for name in value:
                df[name] = 1

### ---------------------------------------------------------------------------------------------------------
def TRANSFORM_(data):
    global condition

    # Initialize empty arrays to store the values
    Arr1 = []
    Arr2 = []
    Arr3 = []

    for key, value in data.items():
        ### To read the file used
        if str('TRANSFORM').lower() in str(key).lower():
            lines = '\n'.join(value)
            lines = lines.split("\n")

            # Loop through the lines and extract the values
            for line in lines:
                line_values = line.split('=')  # split the line into values
                print(line_values)
                var_name = line_values[0].strip()  # extract the variable name
                
                var_value = line_values[1].strip().split()[0]  # extract the variable value

                            
                # Regular expression pattern to extract the condition
                pattern = r'^\w+=\([^\)]*\)(?:\s+(.*?)(?:\n|$))?$'

                # Split the string into lines and extract the condition from each line
                conditions = []

                for line in line.split('\n'):
                    match = re.match(pattern, line)
                 
                    if match:
                        condition = match.group(1)
                        
                        if condition is not None:
                            conditions.append(condition.strip())
                        else:
                            conditions.append(None)

                # Append the values to the arrays
                Arr1.append(var_name)
                Arr2.append(var_value)

                Arr3.append(condition)

    return Arr1, Arr2, Arr3
        
### ---------------------------------------------------------------------------------------------------------
def EST_RSE_(data):
    for key, value in data.items():
        pass

df = FILTER_(data, df)
GROUP_(data, df)
TRANSFORM_(data)
RENAME_(data, df)
NEWVARIABLE_(data, df)
FILE_(data)



['ST_GR', '("G1")   ST in ("10,19,20,21")']
['ST_GR', '("G2")   ST in ("11,12,13,14,15,16,17,18")']
['ST_GR', '("G3")   Otherwise']
['POP', '(1)']
['WRK', '(1)  CWS in ("11,12,21,31,41,51,61,61,71,72")']
['WRK', '(0)  CWS NOT in ("11,12,21,31,41,51,61,61,71,72")']
['LF', '(1)   CWS in ("11,12,21,31,41,51,61,61,71,72,81")']
['LF', '(0)   Otherwise']


In [8]:
GROUP_(data, df)

['SEC', 'ST_GR']

In [9]:
EST_RSE_(data)

In [10]:
def parse_query_string(query_string):
    # Replace curly quotes with straight quotes
    query_string = query_string.replace("”", "\"")
    
    if 'in' in query_string:
        if ('NOT in' or 'not in') in query_string:
            query_string = re.sub(r'(\b\w+\b)\s+NOT in\s*\(([^)]+)\)', r'~\1.isin([\2])', query_string)
            query_string = re.sub(r'(\b\w+\b)\s+in\s*\(([^)]+)\)', r'\1.isin([\2])', query_string)
        elif 'in' in query_string:
            # Replace 'in' operators with 'isin' and add list brackets
            query_string = re.sub(r'(\b\w+\b)\s+in\s*\(([^)]+)\)', r'\1.isin([\2])', query_string)
        else:
            pass
            
    else:
        query_string = re.sub(r'''(\b\w+\b)(\s*[<>=!]+\s*)('[^']*'|"[^"]*")|(\b\w+\b)(\s*[<>=!]+\s*)("[^]*'|"[^"]*")''', r'\1\2\3', query_string)
        query_string = re.sub(r'''(\b\w+\b)(\s*[<>=!]+\s*)(\d+)''', r'\1\2\3', query_string)

    # Replace double quotes with single quotes
    query_string = query_string.replace("\"", "'")
    
    # Replace 'AND' with '&', 'OR' with '|', and add parentheses
    query_string = query_string.replace(" AND ", " & ")\
                        .replace(" OR ", " | ").\
                            replace("&&", '&').\
                                replace('||', '|').\
                                    replace('<>', '!=').\
                                        replace('NOT', '~')
    
    # Replace column names with df[column] syntax
    query_string = re.sub(r'(\b\w+\b)=', r'\1==', query_string)


    # # Define a function to replace variable names with df["<variable_name>"]
    # def replace_var(match):
    #     return f'df["{match.group(0)}"]'

    # # Use regex to replace variable names with df["<variable_name>"]
    # pattern = re.compile(r'''\b\w+\b(?=\s*[<>=!]=*\s*[\'"\d])''')
    # new_string = pattern.sub(replace_var, query_string)

    query_string = re.sub(r"'(\d+)'|\"(\d+)\"", lambda match: str(int(match.group(1) or match.group(2))), query_string)
    query_string = re.sub(r'\[(\d+,\s*)+\d+\]', lambda match: '[' + match.group(0).replace("\'", "'") + ']', query_string)
    
    return query_string


final_parsed_query = []

for i in list(TRANSFORM_(data))[2]:
    # print(i)
    if i:
        parsed_query = parse_query_string(i)
        final_parsed_query.append(parsed_query)
    else:
        final_parsed_query.append(i)

### Calling the final_parsed_query function
### ---------------------------------------
final_parsed_query

['ST_GR', '("G1")   ST in ("10,19,20,21")']
['ST_GR', '("G2")   ST in ("11,12,13,14,15,16,17,18")']
['ST_GR', '("G3")   Otherwise']
['POP', '(1)']
['WRK', '(1)  CWS in ("11,12,21,31,41,51,61,61,71,72")']
['WRK', '(0)  CWS NOT in ("11,12,21,31,41,51,61,61,71,72")']
['LF', '(1)   CWS in ("11,12,21,31,41,51,61,61,71,72,81")']
['LF', '(0)   Otherwise']


["ST.isin(['10,19,20,21'])",
 "ST.isin(['11,12,13,14,15,16,17,18'])",
 'Otherwise',
 None,
 "CWS.isin(['11,12,21,31,41,51,61,61,71,72'])",
 "~CWS.isin(['11,12,21,31,41,51,61,61,71,72'])",
 "CWS.isin(['11,12,21,31,41,51,61,61,71,72,81'])",
 'Otherwise']

In [11]:
df['AGE'].unique()

array([ 40,  18,  19,  38,  21,  28,  52,  26,  45,  25,  36,  17,  30,
        55,  44,  16,  20,  68,  35,  42,  39,  65,  43,  15,  60,  48,
        22,  27,  24,  62,  23,  56,  66,  47,  50,  57,  61,  58,  31,
        34,  64,  54,  29,  32,  41,  59,  51,  75,  70,  37,  33,  85,
        72,  53,  73,  46,  67,  49,  80,  63,  78,  71,  82,  81,  69,
        74,  86,  76,  84,  83,  79,  95,  87,  90,  77,  89, 110,  96,
       100,  88,  93,  92,  98,  94, 105,  91,  99,  97, 109])

In [12]:
Arr1 = list(TRANSFORM_(data))[0]
Arr2 = list(TRANSFORM_(data))[1]

# print(Arr2)
Arr2 = [i.strip('()\'\"') for i in Arr2]
Arr3 = final_parsed_query

Arr2 = [float(elem) if elem.replace('.', '', 1).isdigit() else elem for elem in Arr2]
print(Arr2)

# print(Arr1)
# print(Arr2)
# print(Arr3)

['ST_GR', '("G1")   ST in ("10,19,20,21")']
['ST_GR', '("G2")   ST in ("11,12,13,14,15,16,17,18")']
['ST_GR', '("G3")   Otherwise']
['POP', '(1)']
['WRK', '(1)  CWS in ("11,12,21,31,41,51,61,61,71,72")']
['WRK', '(0)  CWS NOT in ("11,12,21,31,41,51,61,61,71,72")']
['LF', '(1)   CWS in ("11,12,21,31,41,51,61,61,71,72,81")']
['LF', '(0)   Otherwise']
['ST_GR', '("G1")   ST in ("10,19,20,21")']
['ST_GR', '("G2")   ST in ("11,12,13,14,15,16,17,18")']
['ST_GR', '("G3")   Otherwise']
['POP', '(1)']
['WRK', '(1)  CWS in ("11,12,21,31,41,51,61,61,71,72")']
['WRK', '(0)  CWS NOT in ("11,12,21,31,41,51,61,61,71,72")']
['LF', '(1)   CWS in ("11,12,21,31,41,51,61,61,71,72,81")']
['LF', '(0)   Otherwise']
['G1', 'G2', 'G3', 1.0, 1.0, 0.0, 1.0, 0.0]


In [13]:
df.head(3)

,SEC,ST,FSU,IDNO,SEG,SSS,SSU,STRM,SSTRM,SS,...,PAS,PS_SS,CWS,ERN_SELF,ERN_REG,ST_GR,STRMID,POP,LF,WRK
0,1,28,17469,1,1,1,1,1,1,1,...,51,51,51,0,0,0,01280101,1,1,1
1,1,28,17469,1,1,1,1,1,1,1,...,91,91,91,0,0,0,01280101,1,1,1
2,1,28,17469,1,1,1,1,1,1,1,...,91,91,91,0,0,0,01280101,1,1,1


In [14]:
set(Arr1) - set(list(df.columns))

set()

In [15]:

# Check for name mismatches and create new columns
mismatched_names = set(Arr1) - set(df.columns)

new_columns = {name: [0] * df.shape[0] for name in mismatched_names}
df = df.assign(**new_columns)

mismatched_names

set()

In [16]:
Arr3

["ST.isin(['10,19,20,21'])",
 "ST.isin(['11,12,13,14,15,16,17,18'])",
 'Otherwise',
 None,
 "CWS.isin(['11,12,21,31,41,51,61,61,71,72'])",
 "~CWS.isin(['11,12,21,31,41,51,61,61,71,72'])",
 "CWS.isin(['11,12,21,31,41,51,61,61,71,72,81'])",
 'Otherwise']

In [17]:
Arr3 = [i.replace("(['", "([").replace("'])", "])") if i is not None else 'True' if Arr3[j] == 'Otherwise' else True for j, i in enumerate(Arr3)]
Arr3

['ST.isin([10,19,20,21])',
 'ST.isin([11,12,13,14,15,16,17,18])',
 'Otherwise',
 True,
 'CWS.isin([11,12,21,31,41,51,61,61,71,72])',
 '~CWS.isin([11,12,21,31,41,51,61,61,71,72])',
 'CWS.isin([11,12,21,31,41,51,61,61,71,72,81])',
 'Otherwise']

In [18]:
new_arr3 = []
for i, value in enumerate(Arr3):
    if value == 'Otherwise':
        other_values = set(['~(' + Arr3[j] + ')' for j in range(len(Arr3)) if Arr1[j] == Arr1[i] and Arr3[j] != 'Otherwise'])
        new_value = ' & '.join(other_values)
        print(new_value)
        new_arr3.append(new_value)
    else:
        new_arr3.append(value)
        print(value)

# new_arr3

ST.isin([10,19,20,21])
ST.isin([11,12,13,14,15,16,17,18])
~(ST.isin([10,19,20,21])) & ~(ST.isin([11,12,13,14,15,16,17,18]))
True
CWS.isin([11,12,21,31,41,51,61,61,71,72])
~CWS.isin([11,12,21,31,41,51,61,61,71,72])
CWS.isin([11,12,21,31,41,51,61,61,71,72,81])
~(CWS.isin([11,12,21,31,41,51,61,61,71,72,81]))


In [19]:
Arr3 = new_arr3

In [20]:
df['WRK'].unique()

array([1])

In [21]:
Arr2

['G1', 'G2', 'G3', 1.0, 1.0, 0.0, 1.0, 0.0]

In [22]:
for i in range(len(Arr2)):
    if type(Arr3[i]) == str:
        print(Arr3[i])

ST.isin([10,19,20,21])
ST.isin([11,12,13,14,15,16,17,18])
~(ST.isin([10,19,20,21])) & ~(ST.isin([11,12,13,14,15,16,17,18]))
CWS.isin([11,12,21,31,41,51,61,61,71,72])
~CWS.isin([11,12,21,31,41,51,61,61,71,72])
CWS.isin([11,12,21,31,41,51,61,61,71,72,81])
~(CWS.isin([11,12,21,31,41,51,61,61,71,72,81]))


In [23]:
# print()
df.query("CWS==31 & GEDU.isin([5,6,7,11,12])")

,SEC,ST,FSU,IDNO,SEG,SSS,SSU,STRM,SSTRM,SS,...,PAS,PS_SS,CWS,ERN_SELF,ERN_REG,ST_GR,STRMID,POP,LF,WRK
69,1,28,17471,4,1,2,2,1,2,1,...,31,31,31,0,7000,0,01280102,1,1,1
90,1,28,17471,8,1,3,2,1,2,1,...,31,31,31,0,4500,0,01280102,1,1,1
208,1,28,17476,3,1,2,2,1,4,2,...,31,31,31,0,8000,0,01280104,1,1,1
228,1,28,17477,1,1,1,1,1,5,1,...,31,31,31,0,40000,0,01280105,1,1,1
255,1,28,17478,1,1,1,1,1,5,2,...,31,31,31,0,6000,0,01280105,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109801,2,34,27427,4,1,2,2,1,3,2,...,31,31,31,0,25000,0,02340103,1,1,1
109815,2,34,27427,7,1,4,1,1,3,2,...,31,31,31,0,7000,0,02340103,1,1,1
109819,2,34,27428,1,1,1,1,1,3,2,...,31,31,31,0,25000,0,02340103,1,1,1
109832,2,34,27428,4,1,2,2,1,3,2,...,31,31,31,0,15000,0,02340103,1,1,1


In [24]:
# Apply the queries in Arr3 to filter the rows of the dataframe
for i, query_str in enumerate(Arr3):
    
    if query_str is True:
        if type(Arr2[i]) == str:
            if '+' in Arr2[i]:
                df[Arr1[i]] = df[Arr2[i].split('+')].apply(lambda x: x.sum(), axis=1)
                print('*** First if', Arr2[i], Arr1[i], Arr3[i], i)

            else:
                df[Arr1[i]] = df[[Arr2[i]]].apply(lambda x: x, axis=1)
                print('*** First else', Arr2[i], Arr1[i], Arr3[i], i)
        # if type(Arr3[i]):
        #     print('^^^ Second if', Arr2[i], Arr1[i], Arr3[i], i)
        #     df.loc[df_query.index, Arr1[i]] = Arr2[i]
    
    else:
        # try:
        # query_str = re.sub(r'0?\s?(\d+)', r'\1', query_str)
        print(query_str)
        df_query = df.query(query_str)
        # except:
        #     pass

        if type(Arr2[i]) == float:
            df.loc[df_query.index, Arr1[i]] = Arr2[i]
            # print('### Third if', Arr2[i], Arr1[i], Arr3[i])
        else:
            df.loc[df_query.index, Arr1[i]] = Arr2[i]
            # print('@@@ First else', Arr2[i], Arr1[i], Arr3[i])


        if type(Arr2[i]) == str:
            if Arr2[i] in list(df.columns):
                df[Arr1[i]] = df[[Arr2[i]]].apply(lambda x: x, axis=1)
                # print('$$$ Fourth if', Arr2[i], Arr1[i], Arr3[i])

# print(df)

ST.isin([10,19,20,21])
ST.isin([11,12,13,14,15,16,17,18])
~(ST.isin([10,19,20,21])) & ~(ST.isin([11,12,13,14,15,16,17,18]))
CWS.isin([11,12,21,31,41,51,61,61,71,72])
~CWS.isin([11,12,21,31,41,51,61,61,71,72])
CWS.isin([11,12,21,31,41,51,61,61,71,72,81])
~(CWS.isin([11,12,21,31,41,51,61,61,71,72,81]))


In [25]:
# # Apply the queries in Arr3 to filter the rows of the dataframe
# for i, query_str in enumerate(Arr3):
    
#     if query_str is True:
#         print(query_str)

#         try:
#             query_str = re.sub(r'(?<=\d)\s+(?=\d)', '', query_str)
#             df_query = df.query(query_str)
#         except:
#             pass

#         if type(Arr2[i]) == str:
#             if '+' in Arr2[i]:
#                 df[Arr1[i]] = df[Arr2[i].split('+')].apply(lambda x: x.sum(), axis=1)
#                 print('*** First if', Arr2[i], Arr1[i], Arr3[i], i)

#             if True:
#                 # try:
#                 query_str = re.sub(r'(?<=\d)\s+(?=\d)', '', query_str)
#                 df_query = df.query(query_str)
#                 # except:
#                 #     pass
#                 df[Arr1[i]] = df[[Arr2[i]]].apply(lambda x: x, axis=1)
#                 print('*** First else', Arr2[i], Arr1[i], Arr3[i], i)

#         else:
#             df.loc[df_query.index, Arr1[i]] = Arr2[i]
#             print('*** 2nd else', Arr2[i], Arr1[i], Arr3[i], i)
    
#     else:
#         query_str = re.sub(r'(?<=\d)\s+(?=\d)', '', query_str)
#         # try:
#         df_query = df.query(query_str)
#         # except:
#         #     pass

#         if type(Arr2[i]) == float:
#             df.loc[df_query.index, Arr1[i]] = Arr2[i]
#             # print('### 2nd if', Arr2[i], Arr1[i], Arr3[i])
#         else:
#             df.loc[df_query.index, Arr1[i]] = Arr2[i]
#             print('@@@ 3rd else', Arr2[i], Arr1[i], Arr3[i])


#         if type(Arr2[i]) == str:
#             if Arr2[i] in list(df.columns):
#                 df[Arr1[i]] = df[[Arr2[i]]].apply(lambda x: x, axis=1)
#                 print('$$$ 3rd if', Arr2[i], Arr1[i], Arr3[i])

# # print(df)

In [26]:
df[['LF']].value_counts()

LF
0     44608
1     39193
dtype: int64

In [27]:
# ### Decide the type of the Variables
# ### --------------------------------
    
# def GROUP_Decide_Type(data):
#     for key, value in data.items():    
#         if str('GROUP').lower() in str(key).lower():
#             value_name = [s.split()[0] for s in value]
#             value_type = [s.split()[1] for s in value]
#             return {k:v for k, v in zip(value_name, value_type)}


# def VARIABLE_Decide_Type(data):
#     for key, value in data.items():    
#         if str('VARIABLE').lower() in str(key).lower():
#             value_name = [s.split()[0] for s in value]
#             value_type = [s.split()[1] for s in value]

#             return {k:v for k, v in zip(value_name, value_type)}


# def RENAME_Decide_Type(data):
#     for key, value in data.items():    
#         if str('RENAME').lower() in str(key).lower():
#             value_name = [s.split()[0] for s in value]
#             value_type = [s.split()[1] for s in value]

#             return {k:v for k, v in zip(value_name, value_type)}

# decide_group = GROUP_Decide_Type(data)
# decide_var = VARIABLE_Decide_Type(data)
# decide_rename = RENAME_Decide_Type(data)


# for k, v in decide_group.items():
#     if v == 'N':
#         print(v)
#         df[k] = pd.to_numeric(df[k])
    
#     if v == 'C' or 'c':
#         df[k] = df[k].astype(str)

# for i, j in decide_var.items():
#     if j == 'N' or 'n':
#         df[i] = pd.to_numeric(df[i])
    
#     if j == 'C' or 'c':
#         df[i] = df[i].astype(str)

# for m, n in decide_rename.items():
#     if n is 'N' or 'n':
#         df[m] = pd.to_numeric(df[m])
    
#     if n is 'C' or 'c':
#         df[m] = df[m].astype(str)

In [28]:
df.to_csv('WorkFile_Balanced_Half_Sample_Method_Example1.csv')

In [29]:
df.columns

Index(['SEC', 'ST', 'FSU', 'IDNO', 'SEG', 'SSS', 'SSU', 'STRM', 'SSTRM', 'SS',
       'NSS', 'NSC', 'MULT', 'HH_SIZE', 'HHTYPE', 'RELG', 'SG', 'MHCE',
       'DC_ALL', 'DC_ST', 'SRL', 'REL', 'SEX', 'MSEX', 'AGE', 'MARST', 'GEDU',
       'PAS', 'PS_SS', 'CWS', 'ERN_SELF', 'ERN_REG', 'ST_GR', 'STRMID', 'POP',
       'LF', 'WRK'],
      dtype='object')

In [30]:
df['SEC'].unique()

array([1, 2])

**STEP 1**

In [31]:
# # Read the data from the CSV file
# df = pd.read_csv('Work_File1A.csv')

# Get the current column names
old_col_names = df.columns.tolist()

# Create a dictionary with the new column names
new_col_names = {col: col.strip() for col in old_col_names}

# Rename the columns using the dictionary
df.rename(columns=new_col_names, inplace=True)

df['MULT'] = pd.to_numeric(df['MULT'])

# Calculate the SS_MULT and CMULT fields
df['SS_MULT'] = df['MULT'] / 100
df['CMULT'] = df.apply(lambda row: row['MULT'] / 200 if row['NSC'] > row['NSS'] else row['MULT'] / 100, axis=1)

lst_df = list(df.columns)

groupby_cols = GROUP_(data, df)

# Create a list of all possible subsets of groupby_cols
groupby_cols_subsets = [list(subset) for i in range(len(groupby_cols) + 1)
                        for subset in itertools.combinations(groupby_cols, i)]

# Select columns that are not in groupby_cols
cols = [col for col in df.columns if col in groupby_cols]

agg_dict = {col: sum for col in cols if (df[col].dtype == int or df[col].dtype == float) and col in groupby_cols}





################
Arr_W1 = []


### Loop over each subset of columns and perform the aggregation
### ------------------------------------------------------------
for i in range(len(groupby_cols_subsets)):

    print(groupby_cols_subsets[i])

    # Define the prefix to use for the new columns
    prefix = 'Z'

    ## For All Combinations
    if i == 0:

        # # If empty list is selected, perform aggregation without grouping
        # w1 = df.agg(agg_dict).to_frame().T

        # print(new_df)

        test_lst = groupby_cols

        df['POP'] = pd.to_numeric(df['POP']) * df['CMULT']
        df['LF'] = pd.to_numeric(df['LF']) * df['CMULT']
        df['WRK'] = pd.to_numeric(df['WRK']) * df['CMULT']
        df['no_sam'] = df.shape[0]

        agg_dict = {'POP': 'sum', 'LF': 'sum', 'WRK': 'sum', 'no_sam': 'size'}

        # Group the data by SEC and ST_GR and calculate the aggregates
        w1 = df.agg(agg_dict).to_frame().T

        w1 = w1.rename(columns={'POP': 'pophat', 'LF': 'lfhat', 'WRK': 'wrkhat' }).reset_index()

        w1['groupby_cols'] = 'All columns'

        new_col_val_ = []

        for i in range(len(list(set(groupby_cols_subsets[-1]) - set(w1.columns)))):
            # Create a new column with the appropriate number of "Z"s and the prefix
            new_col_val_.append(prefix * (i+1))

        for i in range(len(list(set(groupby_cols_subsets[-1]) - set(w1.columns)))):
            # Add the new column to the DataFrame
            w1[test_lst[i]] = new_col_val_[i]

        ################
        Arr_W1.append(w1)
        
    else:
        # Create a new dataframe by grouping by the selected columns and aggregating
        # Note: here I'm just calculating the sum of columns C and D, but you can use any aggregation function
        # new_df = df.groupby(list(groupby_cols_subset)).agg(agg_dict).reset_index()

        w11 = df.groupby(groupby_cols_subsets[i]).agg(
                                                        pophat=('POP', 'sum'),
                                                            lfhat=('LF', 'sum'),
                                                                wrkhat=('WRK', 'sum'),
                                                                    no_sam= ('POP', 'size')
                                                        ).reset_index()

        print(w11)

        # Rename the columns and reset the index
        # w1 = w1.rename(columns={'POP': 'pophat', 'LF': 'lfhat', 'WRK': 'wrkhat'}).reset_index()

        test_lst = groupby_cols_subsets[i]

        # Add a new column to indicate which groupby columns are selected
        w11['groupby_cols'] = ', '.join(groupby_cols_subsets[i])

        new_col_val = []
        for i in range(len(list(set(groupby_cols_subsets[-1]) - set(w11.columns)))):
            # Create a new column with the appropriate number of "Z"s and the prefix
            new_col_val.append(prefix * (i+1))

        for i in range(len(list(set(groupby_cols_subsets[-1]) - set(w11.columns)))):
            # Add the new column to the DataFrame
            w11[list(set(groupby_cols_subsets[-1]) - set(w11.columns))[i]] = new_col_val[i]


        Arr_W1.append(w11)

        
        print(Arr_W1)

[]
['SEC']
   SEC        pophat         lfhat        wrkhat  no_sam
0    1  5.591594e+08  2.699406e+08  2.513465e+08   46719
1    2  2.459235e+08  1.126721e+08  1.032697e+08   37082
[   index        pophat         lfhat        wrkhat   no_sam groupby_cols SEC  \
0      0  8.050829e+08  3.826127e+08  3.546162e+08  83801.0  All columns   Z   

  ST_GR  
0    ZZ  ,    SEC        pophat         lfhat        wrkhat  no_sam groupby_cols ST_GR
0    1  5.591594e+08  2.699406e+08  2.513465e+08   46719          SEC     Z
1    2  2.459235e+08  1.126721e+08  1.032697e+08   37082          SEC     Z]
['ST_GR']
  ST_GR        pophat         lfhat        wrkhat  no_sam
0    G1  1.769427e+08  7.581469e+07  7.050562e+07   14365
1    G2  2.935630e+07  1.435083e+07  1.313345e+07   11786
2    G3  5.987839e+08  2.924472e+08  2.709771e+08   57650
[   index        pophat         lfhat        wrkhat   no_sam groupby_cols SEC  \
0      0  8.050829e+08  3.826127e+08  3.546162e+08  83801.0  All columns   Z   

  

In [32]:
Arr_W1[3]

,SEC,ST_GR,pophat,lfhat,wrkhat,no_sam,groupby_cols
0,1,G1,1.441490e+08,6.177476e+07,5.757861e+07,9158,"SEC, ST_GR"
1,1,G2,2.483506e+07,1.227208e+07,1.126034e+07,7202,"SEC, ST_GR"
2,1,G3,3.901753e+08,1.958938e+08,1.825075e+08,30359,"SEC, ST_GR"
3,2,G1,3.279368e+07,1.403994e+07,1.292701e+07,5207,"SEC, ST_GR"
4,2,G2,4.521249e+06,2.078752e+06,1.873110e+06,4584,"SEC, ST_GR"
5,2,G3,2.086086e+08,9.655337e+07,8.846961e+07,27291,"SEC, ST_GR"


In [33]:
Arr_W1[0]['pophat'].sum(), Arr_W1[1]['pophat'].sum(), Arr_W1[2]['pophat'].sum(), Arr_W1[3]['pophat'].sum()

(805082936.44, 805082936.44, 805082936.4399999, 805082936.44)

In [34]:
df['CMULT'].sum()
Arr_W1[0]

,index,pophat,lfhat,wrkhat,no_sam,groupby_cols,SEC,ST_GR
0,0,8.050829e+08,3.826127e+08,3.546162e+08,83801.0,All columns,Z,ZZ


In [35]:
df['SS'].unique()

array([1, 2])

**STEP 2** IT is running slow, figure it out later...


In [36]:
# Filter dataframe to include only rows where SS is 1 or 2
df = df.query("SS in [1, 2]")
# print(f"Number of rows after filtering: {len(df)}")

Arr_W2 = []

### Loop over subset of columns to perform aggregation
for i in range(len(groupby_cols_subsets)):

    new_groupby_cols_subset = ['STRMID']

    # Define the prefix to use for the new columns
    prefix = 'Z'

    if i == 0:

        df['popstr'] = pd.to_numeric(df['POP']) * df['CMULT']
        df['s1pop'] = np.where(df['SS'] == 1, pd.to_numeric(df['POP']) * df['SS_MULT'], 0)
        df['s2pop'] = np.where(df['SS'] == 2, pd.to_numeric(df['POP']) * df['SS_MULT'], 0)
        df['lfstr'] = pd.to_numeric(df['LF']) * df['CMULT']
        df['s1lf'] = np.where(df['SS'] == 1, pd.to_numeric(df['LF']) * df['SS_MULT'], 0)
        df['s2lf'] = np.where(df['SS'] == 2, pd.to_numeric(df['LF']) * df['SS_MULT'], 0)
        df['wrkstr'] = pd.to_numeric(df['WRK']) * df['CMULT']
        df['s1wrk'] = np.where(df['SS'] == 1, pd.to_numeric(df['WRK']) * df['SS_MULT'], 0)
        df['s2wrk'] = np.where(df['SS'] == 2, pd.to_numeric(df['WRK']) * df['SS_MULT'], 0)

        agg_dict = {'popstr': 'sum', 's1pop': 'sum', 's2pop': 'sum', 'lfstr': 'sum', 's1lf': 'sum', \
                    's2lf': 'sum', 'wrkstr': 'sum', 's1wrk': 'sum', 's2wrk': 'sum'}

        # # If empty list is selected, perform aggregation without grouping
        w2 = df.agg(agg_dict).to_frame().T

        w2['STRMID'] = 'ALL'

        new_col_val_ = []

        for i in range(len(list(set(groupby_cols) - set(w2.columns)))):
            # Create a new column with the appropriate number of "Z"s and the prefix
            new_col_val_.append(prefix * (i+1))

        for i in range(len(list(set(groupby_cols) - set(w2.columns)))):
            # Add the new column to the DataFrame
            w2[groupby_cols[i]] = new_col_val_[i]

        Arr_W2.append(w2)

        continue

    print(groupby_cols_subsets[i])

    new_groupby_cols_subset.extend(groupby_cols_subsets[i])

    print(new_groupby_cols_subset)

    w2 = df.groupby(groupby_cols_subsets[i]).agg(
            popstr=('POP', lambda x: np.sum(x * df['CMULT'])),
            s1pop=('POP', lambda x: np.nansum(np.where(df['SS'] == 1, x * df['SS_MULT'], 0))),
                s2pop=('POP', lambda x: np.nansum(np.where(df['SS'] == 2, x * df['SS_MULT'], 0))),
                lfstr=('LF', lambda x: np.nansum(x * df['CMULT'])),
                    s1lf=('LF', lambda x: np.nansum(np.where(df['SS'] == 1, x * df['SS_MULT'], 0))),
                    s2lf=('LF', lambda x: np.nansum(np.where(df['SS'] == 2, x * df['SS_MULT'], 0))),
                        wrkstr=('WRK', lambda x: np.nansum(x * df['CMULT'])),
                        s1wrk=('WRK', lambda x: np.nansum(np.where(df['SS'] == 1, x * df['SS_MULT'], 0))),
                        s2wrk=('WRK', lambda x: np.nansum(np.where(df['SS'] == 2, x * df['SS_MULT'], 0)))
                            ).reset_index()

    new_col_val = []
    for i in range(len(list(set(groupby_cols_subsets[-1]) - set(w2.columns)))):
        # Create a new column with the appropriate number of "Z"s and the prefix
        new_col_val.append(prefix * (i+1))

    for i in range(len(list(set(groupby_cols_subsets[-1]) - set(w2.columns)))):
        # Add the new column to the DataFrame
        w2[list(set(groupby_cols_subsets[-1]) - set(w2.columns))[i]] = new_col_val[i]

    Arr_W2.append(w2)

['SEC']
['STRMID', 'SEC']
['ST_GR']
['STRMID', 'ST_GR']
['SEC', 'ST_GR']
['STRMID', 'SEC', 'ST_GR']


In [37]:
Arr_W2[0], Arr_W1[0]

(         popstr         s1pop         s2pop         lfstr          s1lf  \
 0  2.325014e+13  2.302328e+13  2.337018e+13  1.122140e+13  1.114522e+13   
 
            s2lf        wrkstr         s1wrk         s2wrk STRMID SEC ST_GR  
 0  1.122295e+13  1.041509e+13  1.040060e+13  1.036422e+13    ALL   Z    ZZ  ,
    index        pophat         lfhat        wrkhat   no_sam groupby_cols SEC  \
 0      0  8.050829e+08  3.826127e+08  3.546162e+08  83801.0  All columns   Z   
 
   ST_GR  
 0    ZZ  )

In [38]:
Arr_W2[1]['popstr'].sum(), Arr_W2[0]['popstr'].sum(), Arr_W2[2]['popstr'].sum(), Arr_W2[3]['popstr'].sum()

# for i in Arr_W1:
#     print(i.columns)

(23250139060018.977,
 23250139060018.977,
 23250139060018.973,
 23250139060018.973)

In [39]:
Arr_W3 = []

# Define the prefix to use for the new columns
prefix = 'Z'

### Loop over subset of columns to perform aggregation
for i in range(len(Arr_W2)):

    # Define the prefix to use for the new columns
    prefix = 'Z'

    # Join w2 and w1 on SEC and ST_GR columns
    # w3 = Arr_W2[i].merge(Arr_W1[i], on=['SEC', 'AGE_GR'])
    w3 = pd.concat([Arr_W2[i], Arr_W1[i]], axis=1)

    # if i==0:
    #     w3['no_sam'] = df.shape[0]
    # elif i>0:
    #     w3['no_sam'] = df.shape[0]
    
    # Add new columns to w3
    w3['r1hat'] = w3['lfhat'] / w3['pophat']
    w3['r2hat'] = w3['wrkhat'] / w3['pophat']

    # Equate s1pop and s2pop
    w3.loc[w3['popstr'] == w3['s1pop'], 's1pop'] = w3.loc[w3['popstr'] == w3['s1pop'], 's2pop']
    w3.loc[w3['popstr'] == w3['s2pop'], 's2pop'] = w3.loc[w3['popstr'] == w3['s2pop'], 's1pop']

    # Equate s1lf and s2lf
    w3.loc[w3['lfstr'] == w3['s1lf'], 's1lf'] = w3.loc[w3['lfstr'] == w3['s1lf'], 's2lf']
    w3.loc[w3['lfstr'] == w3['s2lf'], 's2lf'] = w3.loc[w3['lfstr'] == w3['s2lf'], 's1lf']

    # Equate s1wrk and s2wrk
    w3.loc[w3['wrkstr'] == w3['s1wrk'], 's1wrk'] = w3.loc[w3['wrkstr'] == w3['s1wrk'], 's2wrk']
    w3.loc[w3['wrkstr'] == w3['s2wrk'], 's2wrk'] = w3.loc[w3['wrkstr'] == w3['s2wrk'], 's1wrk']
    
    new_col_val = []
    for i in range(len(list(set(groupby_cols_subsets[-1]) - set(w3.columns)))):
        # Create a new column with the appropriate number of "Z"s and the prefix
        new_col_val.append(prefix * (i+1))

    for i in range(len(list(set(groupby_cols_subsets[-1]) - set(w3.columns)))):
        # Add the new column to the DataFrame
        w3[list(set(groupby_cols_subsets[-1]) - set(w3.columns))[i]] = new_col_val[i]

    Arr_W3.append(w3)




In [40]:
Arr_W3[1]['popstr'].sum(), Arr_W3[0]['popstr'].sum(), Arr_W3[2]['popstr'].sum(), Arr_W3[3]['popstr'].sum()

(23250139060018.977,
 23250139060018.977,
 23250139060018.973,
 23250139060018.973)

In [41]:
Arr_W3[1]

,SEC,popstr,s1pop,s2pop,lfstr,s1lf,s2lf,wrkstr,s1wrk,s2wrk,ST_GR,SEC,pophat,lfhat,wrkhat,no_sam,groupby_cols,ST_GR,r1hat,r2hat
0,1,1.962086e+13,1.963379e+13,1.950255e+13,9.538981e+12,9.582669e+12,9.421381e+12,8.853905e+12,8.956921e+12,8.686241e+12,Z,1,5.591594e+08,2.699406e+08,2.513465e+08,46719,SEC,Z,0.482761,0.449508
1,2,3.629275e+12,3.389497e+12,3.867630e+12,1.682414e+12,1.562547e+12,1.801572e+12,1.561183e+12,1.443676e+12,1.677981e+12,Z,2,2.459235e+08,1.126721e+08,1.032697e+08,37082,SEC,Z,0.458159,0.419926


In [42]:
### Drop duplicate columns
for i in range(len(Arr_W3)):

    Arr_W3[i] = Arr_W3[i].loc[:, ~Arr_W3[i].columns.duplicated()]

    print(Arr_W3[i].columns)

Index(['popstr', 's1pop', 's2pop', 'lfstr', 's1lf', 's2lf', 'wrkstr', 's1wrk',
       's2wrk', 'STRMID', 'SEC', 'ST_GR', 'index', 'pophat', 'lfhat', 'wrkhat',
       'no_sam', 'groupby_cols', 'r1hat', 'r2hat'],
      dtype='object')
Index(['SEC', 'popstr', 's1pop', 's2pop', 'lfstr', 's1lf', 's2lf', 'wrkstr',
       's1wrk', 's2wrk', 'ST_GR', 'pophat', 'lfhat', 'wrkhat', 'no_sam',
       'groupby_cols', 'r1hat', 'r2hat'],
      dtype='object')
Index(['ST_GR', 'popstr', 's1pop', 's2pop', 'lfstr', 's1lf', 's2lf', 'wrkstr',
       's1wrk', 's2wrk', 'SEC', 'pophat', 'lfhat', 'wrkhat', 'no_sam',
       'groupby_cols', 'r1hat', 'r2hat'],
      dtype='object')
Index(['SEC', 'ST_GR', 'popstr', 's1pop', 's2pop', 'lfstr', 's1lf', 's2lf',
       'wrkstr', 's1wrk', 's2wrk', 'pophat', 'lfhat', 'wrkhat', 'no_sam',
       'groupby_cols', 'r1hat', 'r2hat'],
      dtype='object')


In [43]:
Arr_W3[3]['SEC'].value_counts()

1    3
2    3
Name: SEC, dtype: int64

In [44]:
agg_dict

{'popstr': 'sum',
 's1pop': 'sum',
 's2pop': 'sum',
 'lfstr': 'sum',
 's1lf': 'sum',
 's2lf': 'sum',
 'wrkstr': 'sum',
 's1wrk': 'sum',
 's2wrk': 'sum'}

In [45]:
Arr_W4 = []

### Loop over subset of columns to perform aggregation
for i in range(len(Arr_W3)):
    mask = Arr_W3[i].columns.duplicated()
    Arr_W3[i] = Arr_W3[i].loc[:, ~mask]

    gby_list = ['no_sam', 'pophat', 'lfhat', 'wrkhat', 'r1hat', 'r2hat'] + groupby_cols

    print(gby_list)

    w4 = Arr_W3[i].groupby(gby_list).agg(

        var_pop = ('s1pop', lambda x: np.sum((x - Arr_W3[i]['s2pop']) ** 2)),

            mse_R1 = ('s1lf', lambda x: np.nansum(
                        ((x - Arr_W3[i]['s2lf']) ** 2) +
                            ((Arr_W3[i]['r1hat'] ** 2) * ((Arr_W3[i]['s1pop'] - Arr_W3[i]['s2pop']) ** 2) -
                                (2 * Arr_W3[i]['r1hat'] * (x - Arr_W3[i]['s2lf']) * (Arr_W3[i]['s1pop'] - Arr_W3[i]['s2pop'])))
                    )),

                mse_R2 = ('s1wrk', lambda x: np.nansum(
                            ((x - Arr_W3[i]['s2wrk']) ** 2) +
                                ((Arr_W3[i]['r2hat'] ** 2) * ((Arr_W3[i]['s1pop'] - Arr_W3[i]['s2pop']) ** 2) -
                                    (2 * Arr_W3[i]['r2hat'] * (x - Arr_W3[i]['s2wrk']) * (Arr_W3[i]['s1pop'] - Arr_W3[i]['s2pop'])))
                        ),
        
    )).reset_index()

    Arr_W4.append(w4)

    print(Arr_W4[i])

['no_sam', 'pophat', 'lfhat', 'wrkhat', 'r1hat', 'r2hat', 'SEC', 'ST_GR']
    no_sam        pophat         lfhat        wrkhat     r1hat     r2hat SEC  \
0  83801.0  8.050829e+08  3.826127e+08  3.546162e+08  0.475246  0.440472   Z   

  ST_GR       var_pop        mse_R1        mse_R2  
0    ZZ  1.203369e+23  7.590561e+21  3.578653e+22  
['no_sam', 'pophat', 'lfhat', 'wrkhat', 'r1hat', 'r2hat', 'SEC', 'ST_GR']
   no_sam        pophat         lfhat        wrkhat     r1hat     r2hat  SEC  \
0   37082  2.459235e+08  1.126721e+08  1.032697e+08  0.458159  0.419926    2   
1   46719  5.591594e+08  2.699406e+08  2.513465e+08  0.482761  0.449508    1   

  ST_GR       var_pop        mse_R1        mse_R2  
0     Z  2.286106e+23  3.986052e+20  1.123872e+21  
1     Z  1.722297e+22  9.590807e+21  4.481190e+22  
['no_sam', 'pophat', 'lfhat', 'wrkhat', 'r1hat', 'r2hat', 'SEC', 'ST_GR']
   no_sam        pophat         lfhat        wrkhat     r1hat     r2hat SEC  \
0   11786  2.935630e+07  1.435083e+07

In [46]:
Arr_W4[1]

,no_sam,pophat,lfhat,wrkhat,r1hat,r2hat,SEC,ST_GR,var_pop,mse_R1,mse_R2
0,37082,2.459235e+08,1.126721e+08,1.032697e+08,0.458159,0.419926,2,Z,2.286106e+23,3.986052e+20,1.123872e+21
1,46719,5.591594e+08,2.699406e+08,2.513465e+08,0.482761,0.449508,1,Z,1.722297e+22,9.590807e+21,4.481190e+22


In [47]:
Arr_W4[1]['pophat'].sum(), Arr_W4[0]['pophat'].sum(), Arr_W4[2]['pophat'].sum(), Arr_W4[3]['pophat'].sum()

(805082936.44, 805082936.44, 805082936.4399999, 805082936.44)

In [48]:
Arr_W5 = []
for i in range(len(Arr_W4)):
    # Calculate RSEs for each variable in T1
    w5 = Arr_W4[i].assign(
        RSE_POP=lambda x: 100 * (x['var_pop'] ** 0.5) / (2 * x['pophat']),
        RSE_R1=lambda x: 100 * (x['mse_R1'] ** 0.5) / (2 * x['lfhat']),
        RSE_R2=lambda x: 100 * (x['mse_R2'] ** 0.5) / (2 * x['wrkhat'])
    )

    Arr_W5.append(w5)

In [49]:
Arr_W5[1]['pophat'].sum(), Arr_W5[0]['pophat'].sum(), Arr_W5[2]['pophat'].sum(), Arr_W5[3]['pophat'].sum()

(805082936.44, 805082936.44, 805082936.4399999, 805082936.44)

In [50]:
Arr_W5[0]

,no_sam,pophat,lfhat,wrkhat,r1hat,r2hat,SEC,ST_GR,var_pop,mse_R1,mse_R2,RSE_POP,RSE_R1,RSE_R2
0,83801.0,8.050829e+08,3.826127e+08,3.546162e+08,0.475246,0.440472,Z,ZZ,1.203369e+23,7.590561e+21,3.578653e+22,21544.118941,11385.380829,26672.963894


In [51]:
Tables = []

for i in range(len(Arr_W5)):
    gby_list = ['no_sam', 'pophat', 'lfhat', 'wrkhat', 'r1hat', 'r2hat', 'RSE_POP', 'RSE_R1', 'RSE_R2'] + groupby_cols
    
    # assuming the data is loaded into a DataFrame called w5
    T1 = Arr_W5[i][gby_list].copy()

    # add empty columns for sec_desc and st_gr_desc
    T1['sec_desc'] = ''
    T1['st_gr_desc'] = ''

    # apply formatting to columns
    T1['R1'] = 100 * T1['r1hat']
    T1['R2'] = 100 * T1['r2hat']

    gby_list = ['sec_desc', 'st_gr_desc', 'no_sam', 'pophat','lfhat', 'wrkhat', 'RSE_POP', 'R1', 'RSE_R1', 'R2', 'RSE_R2'] + groupby_cols

    # reorder columns
    T1 = T1[gby_list]

    gby_list = ['no_sam', 'pophat','lfhat', 'wrkhat', 'r1hat', 'r2hat', 'RSE_POP', 'RSE_R1', 'RSE_R2'] + groupby_cols

    T1 = Arr_W5[i][gby_list].copy()

    T1['sec_desc'] = ' '
    T1['st_gr_desc'] = ' '

    T1['R1'] = 100 * T1['r1hat']
    T1['R2'] = 100 * T1['r2hat']

    gby_list = ['no_sam', 'pophat','lfhat', 'wrkhat', 'RSE_POP', 'R1', 'RSE_R1', 'R2', 'RSE_R2'] + groupby_cols

    T1 = T1[gby_list]

    Tables.append(T1)
    

print(Tables)
print('\n')

[    no_sam        pophat         lfhat        wrkhat       RSE_POP         R1  \
0  83801.0  8.050829e+08  3.826127e+08  3.546162e+08  21544.118941  47.524631   

         RSE_R1         R2        RSE_R2 SEC ST_GR  
0  11385.380829  44.047162  26672.963894   Z    ZZ  ,    no_sam        pophat         lfhat        wrkhat       RSE_POP         R1  \
0   37082  2.459235e+08  1.126721e+08  1.032697e+08  97211.610553  45.815900   
1   46719  5.591594e+08  2.699406e+08  2.513465e+08  11735.143823  48.276147   

         RSE_R1         R2        RSE_R2  SEC ST_GR  
0   8859.826839  41.992625  16231.374141    2     Z  
1  18139.667176  44.950766  42110.843880    1     Z  ,    no_sam        pophat         lfhat        wrkhat        RSE_POP         R1  \
0   11786  2.935630e+07  1.435083e+07  1.313345e+07   26528.134707  48.885000   
1   14365  1.769427e+08  7.581469e+07  7.050562e+07  161432.752753  42.847025   
2   57650  5.987839e+08  2.924472e+08  2.709771e+08   75370.070824  48.840186   



In [52]:
Tables[1]['pophat'].sum(), Tables[0]['pophat'].sum(), Tables[2]['pophat'].sum(), Tables[3]['pophat'].sum()

(805082936.44, 805082936.44, 805082936.4399999, 805082936.44)

In [53]:
def merge_dataframes(tables):
    merged_df = tables[0]
    for df in tables[1:]:
        merged_df = pd.concat([merged_df, df], axis=0)
    return merged_df

merge_dataframes(Tables)


,no_sam,pophat,lfhat,wrkhat,RSE_POP,R1,RSE_R1,R2,RSE_R2,SEC,ST_GR
0,83801.0,8.050829e+08,3.826127e+08,3.546162e+08,21544.118941,47.524631,11385.380829,44.047162,26672.963894,Z,ZZ
0,37082.0,2.459235e+08,1.126721e+08,1.032697e+08,97211.610553,45.815900,8859.826839,41.992625,16231.374141,2,Z
1,46719.0,5.591594e+08,2.699406e+08,2.513465e+08,11735.143823,48.276147,18139.667176,44.950766,42110.843880,1,Z
0,11786.0,2.935630e+07,1.435083e+07,1.313345e+07,26528.134707,48.885000,7754.333582,44.738082,46535.581585,Z,G2
1,14365.0,1.769427e+08,7.581469e+07,7.050562e+07,161432.752753,42.847025,121460.332210,39.846577,18970.454899,Z,G1
2,57650.0,5.987839e+08,2.924472e+08,2.709771e+08,75370.070824,48.840186,52637.967351,45.254576,44044.962065,Z,G3
0,4584.0,4.521249e+06,2.078752e+06,1.873110e+06,42270.862246,45.977384,29986.951328,41.429033,39288.603679,2,G2
1,5207.0,3.279368e+07,1.403994e+07,1.292701e+07,3579.784203,42.812940,32252.550734,39.419213,29255.218744,2,G1
2,7202.0,2.483506e+07,1.227208e+07,1.126034e+07,39053.067349,49.414335,4859.533561,45.340498,48821.595194,1,G2
3,9158.0,1.441490e+08,6.177476e+07,5.757861e+07,198972.840956,42.854780,141771.796711,39.943801,17154.436566,1,G1


In [55]:
merge_dataframes(Tables).to_csv('Example1__Table(T).csv')

In [ ]:
# T1.to_csv('T1_Balanced_Half_Sample_Method.csv')

In [56]:
T1.columns

Index(['no_sam', 'pophat', 'lfhat', 'wrkhat', 'RSE_POP', 'R1', 'RSE_R1', 'R2',
       'RSE_R2', 'SEC', 'ST_GR'],
      dtype='object')